<a href="https://colab.research.google.com/github/Maddi007-Py/Maddi007-Py-CrimeAnalytics_Clustering/blob/main/Code%20Sections/5.3%20Data%20Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **5.3 Data Cleaning**

In [2]:
import warnings
import pandas as pd
from IPython.display import display, HTML
from google.colab import files

# Suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Read the data from CSV file
url = "https://raw.githubusercontent.com/Maddi007-Py/Maddi007-Py-CrimeAnalytics_Clustering/main/Output_CSV/Target_Dataset.csv"
Data_Preparing_df = pd.read_csv(url, low_memory=False).copy()

print("\n\n")

# Output File Name for HTML summary changed to "5.3 Data Cleaning"
html_output_filename = '/content/5.3 Data Cleaning.html'

# Table to store results
steps_summary = []

# Step 1: Dataset Loading
before_step_1 = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 1: Load Dataset",
    "Before Action": before_step_1,
    "Affected by Action": 0,
    "After Action": before_step_1,
    "Unit": "Rows"
})

# Step 2: Identify and remove true duplicates (excluding '_id')
columns_to_check = [col for col in Data_Preparing_df.columns if col != '_id']
duplicate_count = Data_Preparing_df.duplicated(subset=columns_to_check).sum()
rows_before_dedup = Data_Preparing_df.shape[0]
Data_Preparing_df = Data_Preparing_df.drop_duplicates(subset=columns_to_check, keep='first').copy()
rows_after_dedup = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 2: Remove TRUE DUPLICATE Records",
    "Before Action": rows_before_dedup,
    "Affected by Action": duplicate_count,
    "After Action": rows_after_dedup,
    "Unit": "Rows"
})

# Step 3: Drop rows with null, NaN, or missing data
before_drop_rows = Data_Preparing_df.shape[0]
Data_Preparing_df = Data_Preparing_df.dropna().copy()
after_drop_rows = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 3: Drop Rows with Missing Data",
    "Before Action": before_drop_rows,
    "Affected by Action": before_drop_rows - after_drop_rows,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

# Step 4: Strip leading and trailing spaces from string columns
obj_cols = Data_Preparing_df.select_dtypes(include="object").columns
orig_step4 = Data_Preparing_df[obj_cols].copy()
Data_Preparing_df[obj_cols] = Data_Preparing_df[obj_cols].apply(lambda s: s.str.strip())
affected_step4 = (orig_step4 != Data_Preparing_df[obj_cols]).any(axis=1).sum()
steps_summary.append({
    "Step Taken": "Step 4: Strip Leading/Trailing Spaces",
    "Before Action": after_drop_rows,
    "Affected by Action": affected_step4,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

# Step 5: Remove leading apostrophes from string columns
orig_step5 = Data_Preparing_df[obj_cols].copy()
Data_Preparing_df[obj_cols] = Data_Preparing_df[obj_cols].apply(lambda s: s.str.lstrip("'"))
affected_step5 = (orig_step5 != Data_Preparing_df[obj_cols]).any(axis=1).sum()
steps_summary.append({
    "Step Taken": "Step 5: Remove Leading Apostrophes",
    "Before Action": after_drop_rows,
    "Affected by Action": affected_step5,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

# Step 6: Match rows where 'HOOD_158' is 'NSA' and replace with matching value based on coordinates
nsa_replaced_count = 0
for i, row in Data_Preparing_df[Data_Preparing_df['HOOD_158'] == 'NSA'].iterrows():
    match = Data_Preparing_df[
        (Data_Preparing_df['LONG_WGS84'] == row['LONG_WGS84']) &
        (Data_Preparing_df['LAT_WGS84'] == row['LAT_WGS84']) &
        (Data_Preparing_df['HOOD_158'] != 'NSA')
    ]
    if not match.empty:
        matched_value = match.iloc[0]['HOOD_158']
        Data_Preparing_df.loc[i, 'HOOD_158'] = matched_value
        nsa_replaced_count += 1
steps_summary.append({
    "Step Taken": "Step 6: Match & Replace 'NSA' Values",
    "Before Action": after_drop_rows,
    "Affected by Action": nsa_replaced_count,
    "After Action": after_drop_rows,
    "Unit": "Rows"
})

# Step 7: Remove remaining rows where 'HOOD_158' or 'NEIGHBOURHOOD_158' contains 'NSA'
mask_remaining_nsa = (Data_Preparing_df['HOOD_158'] == 'NSA') | (Data_Preparing_df['NEIGHBOURHOOD_158'] == 'NSA')
remaining_nsa_count = mask_remaining_nsa.sum()
before_removal = Data_Preparing_df.shape[0]
Data_Preparing_df = Data_Preparing_df[~mask_remaining_nsa].copy()
after_removal = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Step 7: Remove Remaining 'NSA' Rows",
    "Before Action": before_removal,
    "Affected by Action": remaining_nsa_count,
    "After Action": after_removal,
    "Unit": "Rows"
})

# Step 8: Format Longitude & Latitude to 7 decimals
orig_long = Data_Preparing_df['LONG_WGS84'].copy()
orig_lat = Data_Preparing_df['LAT_WGS84'].copy()
Data_Preparing_df.loc[:, 'LONG_WGS84'] = Data_Preparing_df['LONG_WGS84'].astype(float).map(lambda x: f"{x:.7f}")
Data_Preparing_df.loc[:, 'LAT_WGS84'] = Data_Preparing_df['LAT_WGS84'].astype(float).map(lambda x: f"{x:.7f}")
affected_step8 = ((orig_long != Data_Preparing_df['LONG_WGS84']) | (orig_lat != Data_Preparing_df['LAT_WGS84'])).sum()
steps_summary.append({
    "Step Taken": "Step 8: Format Longitude & Latitude to 7 Decimals",
    "Before Action": after_removal,
    "Affected by Action": affected_step8,
    "After Action": after_removal,
    "Unit": "Rows"
})

# Final row: Rows Affected in 5.3 Data Cleaning
final_row_count = Data_Preparing_df.shape[0]
steps_summary.append({
    "Step Taken": "Rows Affected in <strong>5.3 Data Cleaning </strong>",
    "Before Action": "Initial Load:<br><strong>" + str(before_step_1) + "</strong>",
    "Affected by Action": "Overall Reduction:<br><strong>" + str(before_step_1 - final_row_count) + "</strong>",
    "After Action": "Final Count:<br><strong>" + str(final_row_count) + "</strong>",
    "Unit": "Rows"
})

# Build HTML Table with styling
html_table = """
<table style='border-collapse: collapse; width: 100%; font-size: 18px;'>
    <thead style='background-color: #4CAF50; color: white;'>
        <tr>
            <th colspan="5" style="text-align: center; font-size: 24px; background-color: #2f4f4f; color: white;">
                5.3 Data Cleaning
            </th>
        </tr>
        <tr>
            <th>Step Taken</th>
            <th>Before Action</th>
            <th>Affected by Action</th>
            <th>After Action</th>
            <th>Unit</th>
        </tr>
    </thead>
    <tbody>
"""
for step in steps_summary:
    html_table += f"""
        <tr style='border: 1px solid #dddddd;'>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Step Taken']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Before Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Affected by Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['After Action']}</td>
            <td style='border: 1px solid #dddddd; padding: 8px;'>{step['Unit']}</td>
        </tr>
    """
# Add final note row spanning all columns with the required note text
note_text = (
    "<strong>Note: "
    "Longitude and Latitude were reduced to 7 Decimal Places as,<br>"
    "• 7 decimal places offer precision of <span style='color: green;'>1.1 cm</span>, precise enough for GPS devices.<br>"
    "• Further granularity adds processing time and energy consumption without real-world benefits.<br>"
    "The final cleaned data has been saved as <span style='color: blue;'> 'Cleaned_Data.csv' </span> for further analysis.</strong>"
)
html_table += f"""
        <tr style='border: 1px solid #dddddd;'>
            <td colspan="5" style='border: 1px solid #dddddd; padding: 8px;'>{note_text}</td>
        </tr>
    </tbody>
</table>
"""

# Save the cleaned data
Data_Preparing_df.to_csv('Cleaned_Data.csv', index=False)

# Display the HTML table
display(HTML(html_table))

# Write the HTML file and trigger download
with open(html_output_filename, 'w', encoding='utf-8') as f:
    f.write(html_table)
files.download(html_output_filename)
print("\n\n")

EmptyDataError: No columns to parse from file